In [7]:
#Import Libraries

import pandas as pd  
import sqlalchemy as sa
import acquire
import sklearn.preprocessing
import wrangle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import env

from scipy import stats
from sklearn.model_selection import train_test_split


#Ignore warnings
import warnings
warnings.filterwarnings("ignore")


### 1. Apply the scalers we talked about in this lesson to your data and visualize the results for the unscaled and scaled distribution .


In [ ]:
x_train_and_validate, x_test = train_test_split(x, random_state=123)
x_train, x_validate = train_test_split(x_train_and_validate)


### 2. Apply the .inverse_transform method to your scaled data. Is the resulting dataset the exact same as the original data?

## Min-Max Scaling


In [ ]:
scaler = sklearn.preprocessing.MinMaxScaler()
# Note that we only call .fit with the training data,
# but we use .transform to apply the scaling to all the data splits.
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_validate_scaled = scaler.transform(x_validate)
x_test_scaled = scaler.transform(x_test)

plt.figure(figsize=(13, 6))
plt.subplot(121)
plt.hist(x_train, bins=25, ec='black')
plt.title('Original')
plt.subplot(122)
plt.hist(x_train_scaled, bins=25, ec='black')
plt.title('Scaled')


## Standard Scaler
 Standardization is a linear transformation of our data such that is looks like the standard normal distribution. That is, it will have a mean of 0 and a standard deviation of 1.

x
′
=
x
−
¯
x
σ
x

 Sometimes this is split into two operations:

 * scaling is dividing each data point by the standard deviation. This causes the resulting dataset to have a standard deviation of 1.
* centering is subtracting the mean from each data point. This causes the resulting dataset to have a mean of 0.



In [ ]:
scaler = sklearn.preprocessing.StandardScaler()
# Note that we only call .fit with the training data,
# but we use .transform to apply the scaling to all the data splits.
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_validate_scaled = scaler.transform(x_validate)
x_test_scaled = scaler.transform(x_test)

plt.figure(figsize=(13, 6))
plt.subplot(121)
plt.hist(x_train, bins=25, ec='black')
plt.title('Original')
plt.subplot(122)
plt.hist(x_train_scaled, bins=25, ec='black')
plt.title('Scaled')


### 3. Read the documentation for sklearn's QuantileTransformer. Use normal for the output_distribution and apply this scaler to your data. Visualize the result of your data scaling.


## RobustScaler

A robust scaler is another linear transformation that follows the same idea as the standard scaler but uses parameters that are more robust to outliers.

x
′
=
x
−
med
(
x
)
IQR
x


### 4. Use the QuantileTransformer, but omit the output_distribution argument. Visualize your results. What do you notice?


In [ ]:
scaler = sklearn.preprocessing.RobustScaler()
# Note that we only call .fit with the training data,
# but we use .transform to apply the scaling to all the data splits.
scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_validate_scaled = scaler.transform(x_validate)
x_test_scaled = scaler.transform(x_test)

plt.figure(figsize=(13, 6))
plt.subplot(121)
plt.hist(x_train, bins=25, ec='black')
plt.title('Original')
plt.subplot(122)
plt.hist(x_train_scaled, bins=25, ec='black')
plt.title('Scaled')


### 5. Based on the work you've done, choose a scaling method for your dataset. Write a function within your prepare.py that accepts as input the train, validate, and test data splits, and returns the scaled versions of each. Be sure to only learn the parameters for scaling from your training data!